In [ ]:
import socket
import threading
from Crypto.Cipher import AES
from config import AES_KEY
from scapy.all import sniff, IP, UDP, Raw
from collections import deque
import hashlib

# Kara liste (tehlikeli IP'ler)
blocked_ips = set()

# AES-256 şifreli dosya çözme fonksiyonu
def decrypt_file(input_file, output_file):
    with open(input_file, 'rb') as f:
        nonce, tag, ciphertext = [f.read(x) for x in (16, 16, -1)]
    cipher = AES.new(AES_KEY, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    with open(output_file, 'wb') as f:
        f.write(plaintext)

# UDP veri transferi için sunucu oluşturuluyor
def handle_udp():
    udp_server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    udp_server.bind(("", 5555)) # Sunucu 5555 portunu dinliyor
    print("UDP sunucusu 5555 portunda dinleniyor...")

    while True:
        # 1 - İlk paket: dosya adı
        file_name, addr = udp_server.recvfrom(4096)
        file_name = file_name.decode()
        print(f"UDP dosya adı: {file_name}")

        # 2 - Dosya verisi
        fragments = []
        while True:
            data, _ = udp_server.recvfrom(4096)

            # Anormal veri kontrolü ve loglama:
            if len(data) > 4096:
                warning = f"! {addr} - Çok büyük UDP paketi ({len(data)} byte) algılandı!"
                print(warning)
                with open("attack_log.txt", "a") as log:
                    log.write(warning + "\n")
            elif data != b'EOF' and len(data) < 3:
                warning = f"! {addr} - Çok küçük UDP paketi ({len(data)} byte) algılandı!"
                print(warning)
                with open("attack_log.txt", "a") as log:
                    log.write(warning + "\n")
            
            if data == b'EOF':
                break
            fragments.append(data)

        #3 - Dosyayı doğru isimle kaydet
        with open(file_name, 'wb') as f:
            for frag in fragments:
                f.write(frag)
        print(f"UDP dosyası alındı: {file_name}")
        
        # 4 - Şifre çözme (AES-256 EAX)
        decrypt_file(file_name, file_name.replace('.enc', ''))
        print(f"UDP dosyası çözüldü: {file_name.replace('.enc', '')}")


# TCP veri transferi için sunucu oluşturuluyor
def handle_tcp():
    tcp_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    tcp_server.bind(("", 5556)) # Sunucu 5556 portunu dinliyor
    tcp_server.listen(5) # 5 bağlantıya kadar bekleme kuyruğu
    print("TCP sunucusu 5556 portunda dinleniyor...")

    while True:
        conn, addr = tcp_server.accept()
        print(f"TCP bağlantı: {addr}")

        file_name = conn.recv(1024).decode()
        with open(file_name, 'wb') as f:
            while True:
                data = conn.recv(1024)
                if not data:
                    break

                # TCP Anomali kontrolü ve loglama:
                if len(data) > 1024:
                    warning = f"{addr} - Çok büyük TCP parçası ({len(data)} byte) algılandı!"
                    print(warning)
                    with open("attack_log.txt", "a") as log:
                        log.write(warning + "\n")
                elif len(data) < 3:
                    warning = f"{addr} - Çok kısa TCP parçası ({len(data)} byte) algılandı!"
                    print(warning)
                    with open("attack_log.txt", "a") as log:
                        log.write(warning + "\n")
                        
                f.write(data)
                
        print(f"TCP dosyası alındı: {file_name}")
        decrypt_file(file_name, file_name.replace('.enc', ''))
        print(f"TCP dosyası çözüldü: {file_name.replace('.enc', '')}")
        conn.close()


# Sniffer anomaly detection (tekrar eden UDP flood kontrolü)
recent_packets = deque(maxlen=200) # Flood kontrolü için artan buffer boyutu

# Sniffer fonksiyonu: Ağdaki UDP flood saldırılarını ve anormal trafiği algılar, 100 tekrar sonrası IP'yi kara listeye ekler
def sniff_packets():
    def packet_callback(packet):
        if packet.haslayer(IP):
            ip_layer = packet[IP]
            src_ip = ip_layer.src
            dst_ip = ip_layer.dst

            # Bloklanan IP'yi kontrol et: loglamayı veya işlemeyi durdur
            if src_ip in blocked_ips:
                return  # Bu IP engellendiği için artık işlem yapmıyoruz!
                
            length = len(packet)
            print(f"[Sniffer] {src_ip} -> {dst_ip} | {length} byte")

            if packet.haslayer(UDP):
                payload = bytes(packet[Raw]) if packet.haslayer(Raw) else b""
                payload_hash = hashlib.md5(payload).hexdigest()

                recent_packets.append((src_ip, length, payload_hash))
                repeat_count = sum(1 for p in recent_packets if p == (src_ip, length, payload_hash))

                # Flood için aynı paket tekrarını kontrol et 100 tekrar varsa IP'yi kara listeye al
                if repeat_count >= 100:
                    warning = f"! [Sniffer] {src_ip} -> {dst_ip} - Tehlikeli Flood Detected: {repeat_count} kez tekrar! IP bloklandı."
                    print(warning)
                    with open("attack_log.txt", "a") as log:
                        log.write(warning + "\n")
                    blocked_ips.add(src_ip)  # 100'den fazlaysa IP'yi kara listeye ekle
                    return 

                if length > 4096:
                    warning = f"! [Sniffer] {src_ip} -> {dst_ip} - Çok büyük UDP paketi ({length} byte)!"
                    print(warning)
                    with open("attack_log.txt", "a") as log:
                        log.write(warning + "\n")
                elif length < 3:
                    warning = f"! [Sniffer] {src_ip} -> {dst_ip} - Çok küçük UDP paketi ({length} byte)!"
                    print(warning)
                    with open("attack_log.txt", "a") as log:
                        log.write(warning + "\n")

    print("[Sniffer] Raw packet dinleme başladı...")
    sniff(prn=packet_callback, store=0)

# Main: 3 thread başlatılır (UDP, TCP, Sniffer)
if __name__ == "__main__":
    udp_thread = threading.Thread(target=handle_udp)
    tcp_thread = threading.Thread(target=handle_tcp)
    sniff_thread = threading.Thread(target=sniff_packets)  # Raw packet sniffer thread

    udp_thread.start()
    tcp_thread.start()
    sniff_thread.start()
    
    udp_thread.join()
    tcp_thread.join()
    sniff_thread.join()

UDP sunucusu 5555 portunda dinleniyor...[Sniffer] Raw packet dinleme başladı...
TCP sunucusu 5556 portunda dinleniyor...

[Sniffer] 142.250.187.170 -> 192.168.0.102 | 120 byte
[Sniffer] 192.168.0.102 -> 142.250.187.170 | 75 byte
[Sniffer] 192.168.0.102 -> 142.250.187.170 | 71 byte
[Sniffer] 142.250.187.170 -> 192.168.0.102 | 67 byte
[Sniffer] 192.168.0.102 -> 192.168.0.1 | 84 byte
[Sniffer] 192.168.0.1 -> 192.168.0.102 | 84 byte
[Sniffer] 192.168.0.102 -> 142.250.187.170 | 71 byte
[Sniffer] 142.250.187.170 -> 192.168.0.102 | 67 byte
[Sniffer] 192.168.0.102 -> 142.250.187.170 | 71 byte
[Sniffer] 142.250.187.170 -> 192.168.0.102 | 67 byte
[Sniffer] 192.168.0.102 -> 142.250.187.170 | 71 byte
[Sniffer] 142.250.187.170 -> 192.168.0.102 | 67 byte
[Sniffer] 192.168.0.102 -> 140.82.114.25 | 55 byte
[Sniffer] 192.168.0.102 -> 142.250.187.170 | 71 byte
[Sniffer] 142.250.187.170 -> 192.168.0.102 | 67 byte
[Sniffer] 140.82.114.25 -> 192.168.0.102 | 66 byte
[Sniffer] 192.168.0.102 -> 142.250.187.17

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[Sniffer] 192.168.0.102 -> 142.251.140.42 | 784 byte
[Sniffer] 142.251.140.42 -> 192.168.0.102 | 54 byte
[Sniffer] 192.168.0.102 -> 216.58.212.10 | 71 byte
[Sniffer] 142.251.140.74 -> 192.168.0.102 | 142 byte
[Sniffer] 216.58.212.10 -> 192.168.0.102 | 69 byte
[Sniffer] 192.168.0.102 -> 157.240.238.60 | 123 byte
[Sniffer] 142.251.140.42 -> 192.168.0.102 | 441 byte
[Sniffer] 192.168.0.102 -> 142.251.140.42 | 939 byte
[Sniffer] 157.240.238.60 -> 192.168.0.102 | 54 byte
[Sniffer] 192.168.0.102 -> 172.217.17.106 | 695 byte
[Sniffer] 192.168.0.102 -> 142.251.140.74 | 54 byte
[Sniffer] 142.251.140.42 -> 192.168.0.102 | 54 byte
[Sniffer] 172.217.17.106 -> 192.168.0.102 | 54 byte
[Sniffer] 157.240.238.60 -> 192.168.0.102 | 125 byte
[Sniffer] 192.168.0.102 -> 157.240.238.60 | 54 byte
[Sniffer] 142.251.140.42 -> 192.168.0.102 | 1466 byte
[Sniffer] 142.251.140.42 -> 192.168.0.102 | 546 byte
[Sniffer] 192.168.0.102 -> 142.251.140.42 | 54 byte
[Sniffer] 192.168.0.102 -> 172.217.17.106 | 252 byte
[Sn

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




[Sniffer] 192.168.0.102 -> 64.7.118.163 | 74 byte
[Sniffer] 192.168.0.102 -> 64.7.118.163 | 74 byte
[Sniffer] 192.168.0.102 -> 64.7.118.163 | 75 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 64.7.118.163 | 75 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 64.7.118.163 | 76 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.163 -> 192.168.0.102 | 1494 byte
[Sniffer] 64.7.118.16

...


[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 104.22.57.144 | 54 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 104.22.57.144 | 54 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 104.22.57.144 | 54 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 104.22.57.144 | 54 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 104.22.57.144 | 54 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 192.168.0.102 -> 104.22.57.144 | 54 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sniffer] 104.22.57.144 -> 192.168.0.102 | 1494 byte
[Sni

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




[Sniffer] 192.168.0.102 -> 108.138.3.93 | 55 byte
[Sniffer] 192.168.0.102 -> 192.168.0.1 | 76 byte
[Sniffer] 192.168.0.102 -> 192.168.0.1 | 76 byte
[Sniffer] 192.168.0.102 -> 142.251.140.66 | 74 byte
[Sniffer] 192.168.0.102 -> 34.110.183.42 | 1292 byte
[Sniffer] 192.168.0.102 -> 34.110.183.42 | 1292 byte
[Sniffer] 192.168.0.102 -> 34.110.183.42 | 168 byte
[Sniffer] 146.75.1.229 -> 192.168.0.102 | 66 byte
[Sniffer] 143.244.197.139 -> 192.168.0.102 | 670 byte
[Sniffer] 192.168.0.102 -> 52.17.136.209 | 411 byte
[Sniffer] 192.168.0.102 -> 52.17.136.209 | 93 byte
[Sniffer] 35.214.136.108 -> 192.168.0.102 | 66 byte
[Sniffer] 142.251.140.66 -> 192.168.0.102 | 162 byte
[Sniffer] 142.251.140.66 -> 192.168.0.102 | 64 byte
[Sniffer] 192.168.0.102 -> 142.251.140.66 | 73 byte
[Sniffer] 142.251.140.66 -> 192.168.0.102 | 611 byte
[Sniffer] 142.251.140.66 -> 192.168.0.102 | 63 byte
[Sniffer] 192.168.0.102 -> 142.251.140.66 | 77 byte
[Sniffer] 34.149.50.64 -> 192.168.0.102 | 65 byte
[Sniffer] 34.110.1

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[Sniffer] 192.168.0.102 -> 172.171.87.38 | 54 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 54 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 66 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 66 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 54 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 1494 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 576 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 54 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 54 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 1494 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 1494 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 1270 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 1239 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 54 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 147 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 1008 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 328 byte
[Sniffer] 172.171.87.38 -> 192.168.0.102 | 366 byte
[Sniffer] 192.168.0.102 -> 172.171.87.38 | 54 byte
[Sniffer] 104.2